# Computes the internal-tide wave drag coefficient at each grid point (JSL formula)

Input: 
- Location: lon, lat

Steps:
1. Get the Coriolis frequency, bottom stratification
2. Sample the topography 
3. Fit the 2D topography with a polynomial sloping surface $H = a + bx + cy + dxy$
4. Remove the polynomial fit and compute the mean-square topography $h^2$ 
5. Compute the drag coefficient, $\frac{1}{2}k_h h^2 N_b$, where $k_h = 2 \pi / (2~km)$ is a constant

Output: 
- Drag coefficients: $\sigma$

In [1]:
%matplotlib inline

import xrft
import math
import time
import scipy.io
import numpy as np
import xarray as xr
import netCDF4 as nc
import cmocean as cm
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.colors as colors

from gsw import f
from matplotlib.colors import LogNorm
from scipy.fftpack import fft2, fftn, fftshift

import dask
#from dask.distributed import Client, progress

import warnings
warnings.filterwarnings('ignore')

Create a “cluster” of a scheduler and workers running on the local machine. <br>
**threads_per_worker**: number of threads per each worker <br>
**n_workers**: number of workers to start

In [2]:
# client = Client(threads_per_worker=4, n_workers=1)
# client

In [3]:
def drag_coeff(lon,lat):
    
    print('Computing drag coefficient at (%.1f°E,%.1f°N)... \n' % (lon,lat))
    kh    = 2 * np.pi / 1e+4
    omega = 2 * np.pi / (12.42*3600)
    f_loc = f(lat)
    N_loc = get_N_clim(lon,lat) 
    print('Local bottom stratification is %.1e rad/s' % N_loc)
    topog = xr.open_dataset('/g/data/nm03/lxy581/synbath/SYNBATH.nc')
    depth = topog.z
    if omega>f_loc and omega<N_loc:
        
        step = 0.25
        
        # sample the topography
        topog_sample = depth.where((topog.lon>lon-step) & (topog.lon<lon+step) & (topog.lat>lat-step) & (topog.lat<lat+step), drop=True)
        lon_sample = topog.lon.where((topog.lon>lon-step) & (topog.lon<lon+step), drop=True)
        lat_sample = topog.lat.where((topog.lat>lat-step) & (topog.lat<lat+step), drop=True)

        # fit the 2D topography with a polynomial sloping surface
        LON, LAT = np.meshgrid(lon_sample, lat_sample, copy=False)
        X = LON.flatten()
        Y = LAT.flatten()
        A = np.array([X*0+1, X, Y, X*Y]).T
        B = np.array(topog_sample).flatten()
        coeff, r, rank, s = np.linalg.lstsq(A, B)
        topog_fit = coeff[0] + coeff[1]*LON + coeff[2]*LAT + coeff[3]*LON*LAT
        
        # instead of removing the mean, subtracting a fitted function
        # topog_sample -= topog_sample.mean(skipna=True)
        topog_sample -= topog_fit
        h_ms = np.nanmean(topog_sample ** 2)
        print('mean-square height   = %3.2e \n' % h_ms)

        # compute the drag coefficient
        print('Computing the drag coeff...')
        sigma = compute_drag_coeff(kh,h_ms,N_loc)
        
    else:
        print('This grid point is poleward of critical latitude.')
        sigma = np.nan

    print('Drag coefficient: %3.2e' % sigma)

    return sigma

In [4]:
def get_N_clim(lon,lat):
    
    Nbot_data = xr.open_dataset('/g/data/nm03/lxy581/WOA18/Nbot_1000m_woa18.nc')

    # Read bottom N and convert the unit to rad/s
    Nbot_1km = Nbot_data.Nbot * 2 * np.pi

    # Use the 2D interpolation to find the bottom N
    N_clim = Nbot_1km.interp(lat=lat,lon=lon).values
    
    return N_clim

In [5]:
def compute_drag_coeff(kh,h_ms,N):
    
    sigma = 0.5 * kh * h_ms * N
    
    return sigma

### Testing cell below

Provide input details for the example: <br>
* lon 30$^{\circ}$W <br>
* lat 50$^{\circ}$N
* tidal frequency, in this case, $\omega_{M_2}$ 

In [6]:
lon = -30
lat = 50

Call the function 

In [7]:
start_time = time.time()
sigma = drag_coeff(lon,lat)
end_time = time.time()
exe_time = float(end_time - start_time)
print("Execution time: %.1f seconds! \n" % exe_time)

Computing drag coefficient at (-30.0°E,50.0°N)... 

0.3.0
Local bottom stratification is 9.2e-04 rad/s
mean-square height   = 1.74e+05 

Computing the drag coeff...
Drag coefficient: 5.06e-02
Execution time: 31.9 seconds! 

